# Internal Datastructure: Bus branch model, Admittance and Jacobian Matrix


This jupyter notebooks explains how to access and interpret the internal datastructure with relevant matrices.

### Internal Datastructure

We use the simple example network from the create_simple tutorial as an example for how to access internal calculation parameters:

<img src="pics/example_network_simple.png">


In [1]:
import pandapower as pp
import pandapower.networks as nw

net = nw.example_simple()
print(net)

This pandapower network includes the following parameter tables:
   - gen (1 elements)
   - shunt (1 elements)
   - trafo (1 elements)
   - switch (8 elements)
   - load (1 elements)
   - line (4 elements)
   - ext_grid (1 elements)
   - bus (7 elements)
   - sgen (1 elements)


First, we run a power flow in this network:

In [4]:
pp.runpp(net)

When a power flow is carried out, the element based grid model is translated into a bus-branch model. That bus-branch model is stored in a data structure that is based on the PYPOWER/MATPOWER casefile (with some extensions). This ppc can be accesed after power flow:

In [15]:
net._ppc

{'baseMVA': 1.0,
 'branch': array([[  0.00000000e+00+0.j        ,   1.00000000e+00+0.j        ,
           4.95867769e-05+0.j        ,   1.19008264e-04+0.j        ,
           5.47391104e+00+0.j        ,   0.00000000e+00+0.j        ,
           2.50000000e+02+0.j        ,   2.50000000e+02+0.j        ,
           1.00000000e+00+0.j        ,   0.00000000e+00+0.j        ,
           1.00000000e+00+0.j        ,  -3.60000000e+02+0.j        ,
           3.60000000e+02+0.j        ,  -6.72726820e+00+0.j        ,
          -7.26270133e+00+0.j        ,   6.73035426e+00+0.j        ,
           1.57034577e+00+0.j        ,   0.00000000e+00+0.j        ,
           0.00000000e+00+0.j        ,   0.00000000e+00+0.j        ,
           0.00000000e+00+0.j        ,   0.00000000e+00+0.j        ,
           0.00000000e+00+0.j        ],
        [  2.00000000e+00+0.j        ,   3.00000000e+00+0.j        ,
           6.10000000e-04+0.j        ,   5.60000000e-04+0.j        ,
           7.64035333e-02+0.j       

For information on how this datastructure is defined, please referr to the MATPOWER documentation.

## Nodal Point Admittance Matrix

The nodal point admittance matrix is saved in the ppc and can be accessed directly:

In [18]:
net._ppc["internal"]["Ybus"].todense()

matrix([[ 2983.23471400-7157.02635809j, -2983.23471400+7159.76331361j,
             0.00000000   +0.j        ,     0.00000000   +0.j        ,
             0.00000000   +0.j        ,     0.00000000   +0.j        ],
        [-2983.23471400+7159.76331361j,  2988.86218083-7379.20987831j,
          -106.71078800 -196.05444021j,     0.00000000   +0.j        ,
             0.00000000   +0.j        ,     0.00000000   +0.j        ],
        [    0.00000000   +0.j        ,   116.43273175 -190.44147337j,
          1606.91082690-1693.08819188j,  -889.60186671 +816.68368091j,
          -711.68149336 +653.34694473j,     0.00000000   +0.j        ],
        [    0.00000000   +0.j        ,     0.00000000   +0.j        ,
          -889.60186671 +816.68368091j,  1027.81021222 -903.21268537j,
             0.00000000   +0.j        ,  -138.20834551  +86.56929539j],
        [    0.00000000   +0.j        ,     0.00000000   +0.j        ,
          -711.68149336 +653.34694473j,     0.00000000   +0.j        ,
  

Note that the nodal point admittance matrix is given in per unit values.

## Jacobian Matrix

The jacobian Matrix J in the last iteration step is also stored in the ppc and can be accessed:

In [36]:
net._ppc["internal"]["J"].todense()

matrix([[  9.54706374e+02,   0.00000000e+00,  -8.62862717e+02,
           0.00000000e+00,  -9.18436564e+01,   0.00000000e+00,
          -9.15340445e+02,   0.00000000e+00,  -1.42353628e+02],
        [  0.00000000e+00,   7.69001830e+03,  -2.33112187e+02,
           0.00000000e+00,   0.00000000e+00,   3.05115863e+03,
          -1.22942473e+01,   0.00000000e+00,   0.00000000e+00],
        [ -8.60744808e+02,  -2.33450642e+02,   1.77711034e+03,
          -6.82914890e+02,   0.00000000e+00,   8.42114490e-01,
           1.64630184e+03,  -7.30869907e+02,   0.00000000e+00],
        [  0.00000000e+00,   0.00000000e+00,  -6.86805194e+02,
           6.86805194e+02,   0.00000000e+00,   0.00000000e+00,
          -7.26415769e+02,   7.28943154e+02,   0.00000000e+00],
        [ -9.18404726e+01,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   9.18404726e+01,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00,   1.42355795e+02],
        [  0.00000000e+00,  -3.11475201e+03,   1.2

The jacobian matrix is also given in per unit values.

## Mapping the Buses

The pandapower indices are not equal to the ppc indices for several reasons. Some buses are fused together in case of closed bus-bus switches and auxiliary buses are created for elements like extended wards or three winding transformers. There is however a mapping between pandapower indices and ppc indices that is created during the conversion to keep track of the dependencies that is also stored in the net:

In [37]:
net._pd2ppc_lookups["bus"]

array([0, 1, 1, 2, 2, 3, 4])

To get a ppc index from the pandapower index, simply call the lookup like this:

In [38]:
pandapower_bus_idx = 3
ppc_index = net._pd2ppc_lookups["bus"][pandapower_bus_idx]
print(ppc_index)

2


As you can see, pandapower bus index 3 corresponds to ppc bus index 2. So if we would like to find the diagonal entry of the Ybus matrix for bus 2, we could now access it with that internal index:

In [45]:
Ybus = net._ppc["internal"]["Ybus"]
int_idx = net._pd2ppc_lookups["bus"][ppc_index]
Ybus[int_idx, int_idx]

(2988.862180834743-7379.2098783063702j)

We can also see that some buses are mapped to the same internal bus, such as bus 1 and bus 2:

In [46]:
print(net._pd2ppc_lookups["bus"][1])
print(net._pd2ppc_lookups["bus"][2])

1
1


That is because buses 1 and 2 are connected by a closed bus-bus switch and are therefore represented internally as the same bus:

In [40]:
net.switch.loc[0]

bus           1
element       2
et            b
type         CB
closed     True
name       None
Name: 0, dtype: object